## Attention 신경망 구현 및 학습 - need remind ! (Part 4 Lec 58-60)

In [1]:
import random
import tensorflow as tf
from konlpy.tag import Okt

## 데이터셋 준비


In [2]:
NUM_WORDS = 2000
MAX_LEN = 64     # 문장 길이

In [3]:
dataset_file = 'dataset/chatbot_data.csv' # acquired from 'http://www.aihub.or.kr' and modified
okt = Okt()

with open(dataset_file, 'r', encoding='utf-8') as file:
    lines = file.readlines()
    seq = [' '.join(okt.morphs(line)) for line in lines]

questions = seq[::2]
answers = ['\t ' + lines for lines in seq[1::2]]

num_sample = len(questions)

perm = list(range(num_sample))
random.seed(0)
random.shuffle(perm)

train_q = list()
train_a = list()
test_q = list()
test_a = list()

for idx, qna in enumerate(zip(questions, answers)):
    q, a = qna
    if perm[idx] > num_sample//5:
        train_q.append(q)
        train_a.append(a)
    else:
        test_q.append(q)
        test_a.append(a)

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=NUM_WORDS,
                                                  filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~')

tokenizer.fit_on_texts(train_q + train_a)

train_q_seq = tokenizer.texts_to_sequences(train_q)
train_a_seq = tokenizer.texts_to_sequences(train_a)

test_q_seq = tokenizer.texts_to_sequences(test_q)
test_a_seq = tokenizer.texts_to_sequences(test_a)

x_train = tf.keras.preprocessing.sequence.pad_sequences(train_q_seq,
                                                        value=0,
                                                        padding='pre',
                                                        maxlen=64)
y_train = tf.keras.preprocessing.sequence.pad_sequences(train_a_seq,
                                                        value=0,
                                                        padding='post',
                                                        maxlen=65)


x_test = tf.keras.preprocessing.sequence.pad_sequences(test_q_seq,
                                                       value=0,
                                                       padding='pre',
                                                       maxlen=64)
y_test = tf.keras.preprocessing.sequence.pad_sequences(test_a_seq,
                                                       value=0,
                                                       padding='post',
                                                       maxlen=65)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32).prefetch(1024)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(1).prefetch(1024)

## Encoder

In [4]:
class Encoder(tf.keras.Model):
    def __init__(self):
        super(Encoder, self).__init__()
        
        self.emb = tf.keras.layers.Embedding(NUM_WORDS, MAX_LEN)
        self.lstm = tf.keras.layers.LSTM(512, return_sequences=True, return_state=True) # return_sequences : sequences 형태로 출력

    def call(self, x, training=False, mask=None):
        x = self.emb(x)
        H, h, c = self.lstm(x)   # output, hidden state, cell state
        return H, h, c

## Decoder

In [5]:
class Decoder(tf.keras.Model):
    def __init__(self):
        super(Decoder, self).__init__()
        
        self.emb = tf.keras.layers.Embedding(NUM_WORDS, MAX_LEN)
        self.lstm = tf.keras.layers.LSTM(512, return_sequences=True, return_state=True)
        
        # !!! - Attention layer
        self.attention = tf.keras.layers.Attention()
        
        self.dense = tf.keras.layers.Dense(NUM_WORDS, activation='softmax')

    def call(self, inputs, training=False, mask=None):
        x, s0, c0, H = inputs   # y_ : shifted output, s : hidden state of decoder, c : cell state of decoder
        
        x = self.emb(x)
        S, h, c = self.lstm(x, initial_state=[s0, c0])  # S : all hidden states
        
        S_ = tf.concat([s0[:, tf.newaxis, :], S[:, :-1, :]], axis=1)  # 2D --> 3D
        A = self.attention([S_, H])    # key, value
        y = tf.concat([S, A], axis=-1)
        
        return self.dense(y), h, c

## Seq2seq

In [6]:
class Seq2seq(tf.keras.Model):
    def __init__(self, sos, eos):
        super(Seq2seq, self).__init__()
        
        self.enc = Encoder()
        self.dec = Decoder()
        self.sos = sos   # start of sequence
        self.eos = eos   # end

    def call(self, inputs, training=False, mask=None):
        if training is True:
            x, y = inputs
            H, h, c = self.enc(x)   # output, hidden state, cell state
            y, _, _ = self.dec((y, h, c, H))
            return y
        else:
            x = inputs
            H, h, c = self.enc(x)
            
            y = tf.convert_to_tensor(self.sos)
            y = tf.reshape(y, (1, 1))

            seq = tf.TensorArray(tf.int32, 64)

            for idx in tf.range(64):
                y, h, c = self.dec([y, h, c, H])
                y = tf.cast(tf.argmax(y, axis=-1), dtype=tf.int32)
                y = tf.reshape(y, (1, 1))
                seq = seq.write(idx, y)

                if y == self.eos:
                    break

            return tf.reshape(seq.stack(), (1, 64))

## 학습, 테스트 루프 정의

In [7]:
@tf.function
def train_step(model, inputs, labels, loss_object, optimizer, train_loss, train_accuracy):
    output_labels = labels[:, 1:]
    shifted_labels = labels[:, :-1]
    
    with tf.GradientTape() as tape:
        predictions = model([inputs, shifted_labels], training=True)
        loss = loss_object(output_labels, predictions)
        
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(output_labels, predictions)

@tf.function
def test_step(model, inputs):
    return model(inputs, training=False)

## 학습 환경 정의
### 모델 생성, 손실함수, 최적화 알고리즘, 평가지표 정의

In [8]:
# Create model
model = Seq2seq(sos=tokenizer.word_index['\t'],
                eos=tokenizer.word_index['\n'])

# Define loss and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Define performance metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

## 학습 루프 동작

In [9]:
EPOCHS = 200

for epoch in range(EPOCHS):
    for seqs, labels in train_ds:
        train_step(model, seqs, labels, loss_object, optimizer, train_loss, train_accuracy)

    if (epoch + 1)%10 == 0:
        template = 'Epoch {}, Loss: {:.5f}, Accuracy: {:.5f}'
        print(template.format(epoch + 1,
                              train_loss.result(),
                              train_accuracy.result()))

    train_loss.reset_states()
    train_accuracy.reset_states()

Epoch 10, Loss: 0.47699, Accuracy: 0.91490
Epoch 20, Loss: 0.39943, Accuracy: 0.92751
Epoch 30, Loss: 0.35824, Accuracy: 0.93167
Epoch 40, Loss: 0.31526, Accuracy: 0.93542
Epoch 50, Loss: 0.26386, Accuracy: 0.94153
Epoch 60, Loss: 0.20888, Accuracy: 0.95277
Epoch 70, Loss: 0.14249, Accuracy: 0.96918
Epoch 80, Loss: 0.09356, Accuracy: 0.98030
Epoch 90, Loss: 0.05599, Accuracy: 0.98982
Epoch 100, Loss: 0.03631, Accuracy: 0.99370
Epoch 110, Loss: 0.02129, Accuracy: 0.99663
Epoch 120, Loss: 0.01440, Accuracy: 0.99836
Epoch 130, Loss: 0.00973, Accuracy: 0.99886
Epoch 140, Loss: 0.00709, Accuracy: 0.99902
Epoch 150, Loss: 0.00571, Accuracy: 0.99910
Epoch 160, Loss: 0.00453, Accuracy: 0.99930
Epoch 170, Loss: 0.00771, Accuracy: 0.99898
Epoch 180, Loss: 0.00386, Accuracy: 0.99926
Epoch 190, Loss: 0.00321, Accuracy: 0.99910
Epoch 200, Loss: 0.00300, Accuracy: 0.99902


## 테스트 루프

In [10]:
for test_seq, test_labels in test_ds:
    prediction = test_step(model, test_seq)
    test_text = tokenizer.sequences_to_texts(test_seq.numpy())
    gt_text = tokenizer.sequences_to_texts(test_labels.numpy())
    texts = tokenizer.sequences_to_texts(prediction.numpy())
    print('_')
    print('q: ', test_text)
    print('a: ', gt_text)
    print('p: ', texts)

_
q:  ['여기 기프티콘 되죠 \n']
a:  ['\t 네 현금영수증 해드릴까 요 \n']
p:  ['아메리카노 기프티콘 사용 되었습니다 \n']
_
q:  ['네 에 테이크 아웃 도 가능한가요 \n']
a:  ['\t 네 로 오시 면 테이크 아웃 잔 에 담아 드려요 \n']
p:  ['네 네 까지 드릴게요 \n']
_
q:  ['아메리카노 톨 사이즈 로 주세요 \n']
a:  ['\t 따뜻한 거 로 드릴 까요 \n']
p:  ['네 시럽 넣어 드릴 까요 \n']
_
q:  ['진동 을 따로 주시나요 \n']
a:  ['\t 주 번호 로 드리겠습니다 \n']
p:  ['네 카드 받았습니다 \n']
_
q:  ['자리 있나요 \n']
a:  ['\t 네 있습니다 \n']
p:  ['네 이 쪽 에 있습니다 \n']
_
q:  ['그럼 루이보스 밀크 티 하나 \n']
a:  ['\t 네 알겠습니다 \n']
p:  ['네 가능합니다 \n']
_
q:  ['다음 에 무료 로 하고 엔 도장 찍어주세요 \n']
a:  ['\t 네 \n']
p:  ['네 알겠습니다 \n']
_
q:  ['아메리카노 한 잔 에 얼마 죠 \n']
a:  ['\t 입니다 \n']
p:  ['5천 원 입니다 \n']
_
q:  ['얼마나 \n']
a:  ['\t 바로 만들어 드릴게요 \n']
p:  ['자리 정도 걸려요 \n']
_
q:  ['카푸치노 는 로 주시 고 아메리카노 는 로 \n']
a:  ['\t 네 더 없으세요 \n']
p:  ['드시고 가시나요 \n']
_
q:  ['아메리카노 는 어떤 종류 가 있나요 \n']
a:  ['\t 디카 페인 과 기본 아메리카노 2 종류 있습니다 \n']
p:  ['아뇨 캐리어 는 2 개 이상 부터 가능해요 \n']
_
q:  ['카카오 페이 로 결제 가능한가요 \n']
a:  ['\t 네 가능합니다 \n']
p:  ['물론 이 죠 \n']
_
q:  ['오늘 의 커피 는 커피 로 하나요 맛 이 \n']
a:  ['\t 아 네 오늘 은 과테말라 커피